# Socio-economic indicators preprocessing
Prepare all spatial-explicit socio-economic indicators to be linked with the flood disaster

In [ ]:
# Import modules
import os.path
import requests
import geopandas as gpd
from contextlib import contextmanager  
import rasterio
from rasterio import Affine
from rasterio.enums import Resampling
import numpy as np
import rasterio 
from osgeo import gdal
from osgeo import ogr
from osgeo import gdalconst
import gdal
import xarray as xr

# Create paths
local_path = os.getcwd() + '/'

def mkdir(dir):
    
    if not os.path.exists(dir):
        os.mkdir(dir)
        
data_path = local_path + 'data/'
data_processed_path = local_path + 'data_processed/'
socio_economic_variables_path = data_processed_path + 'socio_economic_variables/'
socio_economic_variables_path_creator = mkdir(socio_economic_variables_path)
forest_data_path = socio_economic_variables_path + 'forest/'
forest_data_path_creator = mkdir(forest_data_path)
forest_path = socio_economic_variables_path + 'forest/'
forest_path_creator = mkdir(forest_path)
urbanization_path = socio_economic_variables_path + 'urbanization/'
urbanization_path_creator = mkdir(urbanization_path)
landuse_path = socio_economic_variables_path + 'landuse/'
landuse_path_creator = mkdir(landuse_path)
FLOPROS_path = socio_economic_variables_path + 'FLOPROS/'
FLOPROS_path_creator = mkdir(FLOPROS_path)
HDI_path = socio_economic_variables_path + 'HDI/'
HDI_path_creator = mkdir(HDI_path)
GDP_path = socio_economic_variables_path + 'GDP/'
GDP_path_creator = mkdir(GDP_path)
GDPpc_path = socio_economic_variables_path + 'GDPpc/'
GDPpc_path_creator = mkdir(GDPpc_path)
female_path = socio_economic_variables_path + 'female/'
female_path_creator = mkdir(female_path)
age_path = socio_economic_variables_path + 'age/' # Used in other script
age_path_creator = mkdir(age_path)
demo_path = socio_economic_variables_path + 'demographic/' # Used in other script
demo_path_creator = mkdir(demo_path)

crs = {'init' :'epsg:4326'} 

# Global forest cover

In [ ]:
# Forest cover tiles need to be first downloaded from "https://glad.umd.edu/Potapov/TCC_2010/treecover2010/".
# Start downloading data
for lat in ['N','S']:
    
    for long in ['E','W']:
   
        for lat_deg in range(0,8+1):
        
            for long_deg in range(0,17+1):
                
                if long_deg <10:
                    long_deg = '0'+str(long_deg)

                url = f"https://glad.umd.edu/Potapov/TCC_2010/treecover2010/treecover2010_{lat_deg}0{lat}_{long_deg}0{long}.tif"
                tif_name = url.rsplit('/', 1)[1]
               
                print(tif_name)
                    
                if os.path.isfile(forest_data_path + tif_name):
                    print ("File exist")
                else:
                    print ("File not exist")
  
                    r = requests.get(url, allow_redirects=True)
                    tif_name=url.rsplit('/', 1)[1]
                    open(forest_data_path+tif_name, 'wb').write(r.content)

In [ ]:
# Resample from 30 m to 300 m to reduce the file size
for lat in ['N','S']:
    
    for long in ['E','W']:
   
        for lat_deg in range(0,8+1):
        
            for long_deg in range(0,17+1):
                
                if long_deg <10:
                    long_deg = '0'+str(long_deg)

                url = f"https://glad.umd.edu/Potapov/TCC_2010/treecover2010_{lat_deg}0{lat}_{long_deg}0{long}.tif"
                tif_name = url.rsplit('/', 1)[1]
                
                print(tif_name)
                
                if os.path.isfile(forest_path + 'resampled_' + tif_name):
                    print ("File exists")
                else:
                    print ("File not exists")
                    
                    try:
                        observed_flooding = path_forests + tif_name
                        raster = gdal.Open(observed_flooding)
                        gt =raster.GetGeoTransform()
                        pixelSizeX_satellite = gt[1]
                        pixelSizeY_satellite =-gt[5]
                        satellite_cropped_resampled_1 = forest_path + 'resampled_' + tif_name
                        pixel_transformerX_satellite = 1/10
                        pixel_transformerY_satellite = 1/10

                        @contextmanager
                        def resample_raster(raster, scaleX, scaleY):
                            t = raster.transform

                            # rescale the metadata
                            transform = Affine(t.a / scaleX, t.b, t.c, t.d, t.e / scaleY, t.f)
                            height = raster.height * scaleY
                            width = raster.width * scaleX

                            profile = src.profile
                            profile.update(transform=transform, driver='GTiff', height=height, width=width, crs=crs)

                            data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
                                    out_shape=(int(raster.count), int(height), int(width)),
                                    resampling=Resampling.average)

                            data = np.float32(data)
                            profile['dtype'] = 'float32'

                            with rasterio.open((satellite_cropped_resampled_1),'w', **profile) as dst:
                                dst.write(data)
                                yield data

                        # Apply Function by defining scaling factor        

                        with rasterio.open(observed_flooding) as src:
                            with resample_raster(src, pixel_transformerX_satellite, pixel_transformerY_satellite) as resampled: 
                                print('Sat Orig dims: {}, New dims: {}'.format(src.shape, resampled.shape)) 
                              
                    except:

                        print('...but sea!')

# FLOPROS

In [ ]:
# FLOPROS data is provided on a province (state) level, this section 'burns' the flood protection information on a grid
# Prepare grid with 0s.
rst_fn = rasterio.open(data_path + 'templates/grid_blank.tif')
raster_profile = rst_fn.profile
raster_profile['count'] = 1
raster_profile['dtype'] = 'float32'
rst_fn = np.squeeze(rst_fn.read())
rst_fn[rst_fn != 777] = 0

with rasterio.open(data_path + 'grid_blank_0.tif', 'w', **raster_profile) as dst:
    dst.write_band(1, rst_fn)   
    
#This raster is the model for our output (CRS, extent)
ndsm = data_path + 'grid_blank_0.tif'

#This shapefile contains the features we want to burn
shp = data_path + 'FLOPROS_shp_V1/FLOPROS_shp_V1.shp'

In [ ]:
# Burn shape into raster - MERGED FLOPROS
data = gdal.Open(ndsm, gdalconst.GA_ReadOnly)
geo_transform = data.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize
vec = ogr.Open(shp)
lyr = vec.GetLayer(0)
pixel_width = geo_transform[1]

output = FLOPROS_path + 'FLOPROS_merged.tif'
target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_UInt32)
target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
band = target_ds.GetRasterBand(1)
NoData_value = 0
band.SetNoDataValue(NoData_value)
band.FlushCache()

driver = ogr.GetDriverByName('ESRI Shapefile')
for feat in lyr:
    burn_value = feat.GetField("MerL_Riv")
    datasource = driver.CreateDataSource(FLOPROS_path + 'temp.shp')
    layer = datasource.CreateLayer('temp', lyr.GetSpatialRef(), geom_type=ogr.wkbPolygon)
    layer.CreateFeature(feat)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[burn_value])
    datasource.Destroy()

#This makes raster to write to disk
target_ds = None    
raster_fn = np.squeeze(rasterio.open(FLOPROS_path + 'FLOPROS_merged.tif').read())
raster_fn = np.float32(raster_fn)
raster_fn = np.fliplr(np.flip(raster_fn))

with rasterio.open(FLOPROS_path + 'FLOPROS_merged.tif', 'w', **raster_profile) as dst:
    dst.write_band(1, raster_fn)  

In [ ]:
# Burn shape into raster - MODELED FLOPROS
data = gdal.Open(ndsm, gdalconst.GA_ReadOnly)
geo_transform = data.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * data.RasterXSize
y_min = y_max + geo_transform[5] * data.RasterYSize
x_res = data.RasterXSize
y_res = data.RasterYSize
vec = ogr.Open(shp)
lyr = vec.GetLayer(0)
pixel_width = geo_transform[1]

output = FLOPROS_path + 'FLOPROS_modeled.tif'
target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_UInt32)
target_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
band = target_ds.GetRasterBand(1)
NoData_value = 0
band.SetNoDataValue(NoData_value)
band.FlushCache()

driver = ogr.GetDriverByName('ESRI Shapefile')
for feat in lyr:
    burn_value = feat.GetField("ModL_Riv")
    datasource = driver.CreateDataSource(FLOPROS_path + 'temp.shp')
    layer = datasource.CreateLayer('temp', lyr.GetSpatialRef(), geom_type=ogr.wkbPolygon)
    layer.CreateFeature(feat)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[burn_value])
    datasource.Destroy()

#This makes raster to write to disk
target_ds = None    
raster_fn = np.squeeze(rasterio.open(FLOPROS_path + 'FLOPROS_modeled.tif').read())
raster_fn = np.float32(raster_fn)
raster_fn = np.fliplr(np.flip(raster_fn))

with rasterio.open(FLOPROS_path + 'FLOPROS_modeled.tif', 'w', **raster_profile) as dst:
    dst.write_band(1, raster_fn) 

# Urbanization

In [ ]:
# Extract year bands from .nc and save as .tif
urb = rasterio.open('netcdf:' +  data_path + 'socio_economic_variables/urbanization/landuse-urbanareas_histsoc_annual_1901_2019.nc')
urb_profile = urb.profile
urb_profile['driver'] = 'GTiff'
urb_profile['count'] = 1
urb_profile['dtype'] = 'float32'
urb_profile['crs'] = crs 

for year,year_val in zip(range(1901,2019),range(1,119)):
    
    if year >= 2000:    
        urb_temp = urb.read(year_val).astype(np.float32) 

        with rasterio.open(urbanization_path + f"urbanization_{year}.tif", 'w', **urb_profile) as dst:
            dst.write_band(1, urb_temp)  

# Landuse

In [ ]:
# Extract year bands from .nc and save as .tif
landuse = rasterio.open('netcdf:' +  data_path + 'socio_economic_variables/landuse/landuse-totals_histsoc_annual_1901_2019.nc')
landuse_profile = landuse.profile
landuse_profile['driver'] = 'GTiff'
landuse_profile['count'] = 1
landuse_profile['dtype'] = 'float32'
landuse_profile['crs'] = crs 

landuse = xr.open_dataset(data_path + 'socio_economic_variables/landuse/landuse-totals_histsoc_annual_1901_2019.nc',decode_times = False)

for year,year_val in zip(range(1901,2019),range(1,119)):
    
    if year >= 2000:   
        landuse_temp = landuse.cropland_total[year_val].values 

        with rasterio.open(landuse_path + f"landuse_{year}.tif", 'w', **landuse_profile) as dst:
            dst.write_band(1, landuse_temp)  

# HDI

In [ ]:
# Extract year bands from .nc and save as .tif
HDI = rasterio.open('netcdf:' +  data_path + 'socio_economic_variables/HDI/HDI_1990_2015_v2.nc')
HDI_profile = HDI.profile
HDI_profile['driver'] = 'GTiff'
HDI_profile['count'] = 1
HDI_profile['dtype'] = 'float32'
HDI_profile['crs'] = crs 

for year,year_val in zip(range(1990,2016),range(1,27)):
    
    if year >= 2000:    
        HDI_temp = HDI.read(year_val).astype(np.float32) 

        with rasterio.open(HDI_path + f"HDI_{year}.tif", 'w', **HDI_profile) as dst:
            dst.write_band(1, HDI_temp)  
            
# Extrapolation of missing years
HDI_2000 = rasterio.open(HDI_path + f"HDI_2000.tif")
HDI_2015 = rasterio.open(HDI_path + f"HDI_2015.tif")
HDI_profile = HDI_2015.profile
HDI_2000_np = np.float32(HDI_2000.read(1))
HDI_2015_np = np.float32(HDI_2015.read(1))
HDI_gradient = np.float32(HDI_2015_np - HDI_2000_np)/15
   
# 2016

HDI_temp_np = np.float32(HDI_2015_np + HDI_gradient * 1)
HDI_temp_np[HDI_temp_np > 1] = 1

with rasterio.open(HDI_path + f"HDI_2016.tif", 'w', **HDI_profile) as dst:
    dst.write_band(1, HDI_temp_np) 
    
# 2017

HDI_temp_np = np.float32(HDI_2015_np + HDI_gradient * 2)
HDI_temp_np[HDI_temp_np > 1] = 1

with rasterio.open(HDI_path + f"HDI_2017.tif", 'w', **HDI_profile) as dst:
    dst.write_band(1, HDI_temp_np) 
    
# 2018

HDI_temp_np = np.float32(HDI_2015_np + HDI_gradient * 3)
HDI_temp_np[HDI_temp_np > 1] = 1

with rasterio.open(HDI_path + f"HDI_2018.tif", 'w', **HDI_profile) as dst:
    dst.write_band(1, HDI_temp_np) 

# GDP

In [ ]:
# Extract year bands from .nc and save as .tif. 
GDP = rasterio.open('netcdf:' +  data_path + 'socio_economic_variables/GDP/GDP_PPP_1990_2015_5arcmin_v2.nc')
GDP_profile = GDP.profile
GDP_profile['driver'] = 'GTiff'
GDP_profile['count'] = 1
GDP_profile['dtype'] = 'float32'
GDP_profile['crs'] = crs 

for year,year_val in zip(range(1990,2016),range(1,27)):
    
    if year >= 2000:    
        GDP_temp = GDP.read(year_val).astype(np.float32) 

        with rasterio.open(GDP_path + f"GDP_{year}.tif", 'w', **GDP_profile) as dst:
            dst.write_band(1, GDP_temp)  
            
# Extrapolation of missing years
GDP_2000 = rasterio.open(GDP_path + f"GDP_2000.tif")
GDP_2015 = rasterio.open(GDP_path + f"GDP_2015.tif")
GDP_profile = GDP_2015.profile
GDP_2000_np = np.float32(GDP_2000.read(1))
GDP_2015_np = np.float32(GDP_2015.read(1))
GDP_2015_np[GDP_2015_np < 0] = np.nan
GDP_gradient = np.float32(GDP_2015_np - GDP_2000_np)/15
   
# 2016

GDP_temp_np = np.float32(GDP_2015_np + GDP_gradient * 1)
GDP_temp_np[GDP_temp_np < 0] = 0

with rasterio.open(GDP_path + f"GDP_2016.tif", 'w', **GDP_profile) as dst:
    dst.write_band(1, GDP_temp_np) 
    
# 2017

GDP_temp_np = np.float32(GDP_2015_np + GDP_gradient * 2)
GDP_temp_np[GDP_temp_np < 0] = 0

with rasterio.open(GDP_path + f"GDP_2017.tif", 'w', **GDP_profile) as dst:
    dst.write_band(1, GDP_temp_np) 
    
# 2018

GDP_temp_np = np.float32(GDP_2015_np + GDP_gradient * 3)
GDP_temp_np[GDP_temp_np < 0] = 0

with rasterio.open(GDP_path + f"GDP_2018.tif", 'w', **GDP_profile) as dst:
    dst.write_band(1, GDP_temp_np) 

# GDP per capita

In [ ]:
# Extract year bands from .nc and save as .tif. 
GDPpc = rasterio.open('netcdf:' +  data_path + 'socio_economic_variables/GDPpc/GDP_per_capita_PPP_1990_2015_v2.nc')
GDPpc_profile = GDPpc.profile
GDPpc_profile['driver'] = 'GTiff'
GDPpc_profile['count'] = 1
GDPpc_profile['dtype'] = 'float32'
GDPpc_profile['crs'] = crs 

for year,year_val in zip(range(1990,2016),range(1,27)):
    
    if year >= 2000:    
        GDPpc_temp = GDPpc.read(year_val).astype(np.float32) 

        with rasterio.open(GDPpc_path + f"GDPpc_{year}.tif", 'w', **GDPpc_profile) as dst:
            dst.write_band(1, GDPpc_temp)  
            
# Extrapolation of missing years
GDPpc_2000 = rasterio.open(GDPpc_path + f"GDPpc_2000.tif")
GDPpc_2015 = rasterio.open(GDPpc_path + f"GDPpc_2015.tif")
GDPpc_profile = GDPpc_2015.profile
GDPpc_2000_np = np.float32(GDPpc_2000.read(1))
GDPpc_2015_np = np.float32(GDPpc_2015.read(1))
GDPpc_2015_np[GDPpc_2015_np < 0] = np.nan
GDPpc_gradient = np.float32(GDPpc_2015_np - GDPpc_2000_np)/15
   
# 2016

GDPpc_temp_np = np.float32(GDPpc_2015_np + GDPpc_gradient * 1)
GDPpc_temp_np[GDPpc_temp_np < 0] = 0

with rasterio.open(GDPpc_path + f"GDPpc_2016.tif", 'w', **GDPpc_profile) as dst:
    dst.write_band(1, GDPpc_temp_np) 
    
# 2017

GDPpc_temp_np = np.float32(GDPpc_2015_np + GDPpc_gradient * 2)
GDPpc_temp_np[GDPpc_temp_np < 0] = 0

with rasterio.open(GDPpc_path + f"GDPpc_2017.tif", 'w', **GDPpc_profile) as dst:
    dst.write_band(1, GDPpc_temp_np) 
    
# 2018

GDPpc_temp_np = np.float32(GDPpc_2015_np + GDPpc_gradient * 3)
GDPpc_temp_np[GDPpc_temp_np < 0] = 0

with rasterio.open(GDPpc_path + f"GDPpc_2018.tif", 'w', **GDPpc_profile) as dst:
    dst.write_band(1, GDPpc_temp_np) 